In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()

circles = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]

def renderSegments(segments, cycle):
    description = str(cycle)
    svg = [f'<rect x="0" y="0" width="128" height="128" fill="#ffffff" stroke="#000000" stroke-width="2.0"/>']
    for i in cycle: 
        svg.append(rt.svgText(str(i), circles[i][0], circles[i][1], 4, anchor='middle'))
        svg.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" stroke="#000000" fill="none" />')
    points = set()
    x_min, y_min, x_max, y_max = 128, 128, 0, 0
    seen_already = set()
    for _segment_ in segments:
        x0,y0 = _segment_[0]
        x1,y1 = _segment_[1]
        points.add((x0,y0)), points.add((x1,y1))
        x_min, x_max = min(x_min, x0, x1), max(x_max, x0, x1)
        y_min, y_max = min(y_min, y0, y1), max(y_max, y0, y1)
        svg.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="0.4" />')
        if (x0,y0) not in seen_already: svg.append(f'<circle cx="{x0}" cy="{y0}" r="{1+random.random()}" stroke="#000000" fill="none" stroke-width="0.1" />')
        if (x1,y1) not in seen_already: svg.append(f'<circle cx="{x1}" cy="{y1}" r="{1+random.random()}" stroke="#000000" fill="none" stroke-width="0.1" />')
        seen_already.add((x0,y0)), seen_already.add((x1,y1))
    for i in range(len(cycle)):
        c0,c1,c2 = circles[cycle[i]], circles[cycle[(i+1)%len(cycle)]], circles[cycle[(i+2)%len(cycle)]]
        if (c1[1] - c0[1])*(c2[0] - c1[0]) == (c2[1] - c1[1]) * (c1[0] - c0[0]):
            ...
        else:
            _xy_ = rt.approxThreeCirclesCenter(c0,c1,c2)
            svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="1" stroke="none" fill="#ff0000" />')
    svg.append(rt.svgText(description + f' | segs={len(segments)}' + f' | pts={len(points)}', 5, 123, 8))
    return f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128">' + ''.join(svg) + '</svg>'

svg_header = ['<svg x="0" y="0" viewBox="0 0 128 128" width="512" height="512">']
svg_header.append(f'<rect x="0" y="0" width="128" height="128" fill="#ffffff" />')
svg_circles = []
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    svg_circles.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="{_color_}" fill="none" />')
    svg_circles.append(f'<circle cx="{cx}" cy="{cy}" r="0.2" stroke="#000000" fill="#000000" />')
    svg_circles.append(rt.svgText(str(i), cx, cy-1, 4, anchor='middle'))
svg_footer = ['</svg>']
#rt.tile([''.join(svg_header) + ''.join(svg_circles) + ''.join(svg_footer)])

In [2]:
#
# Point Version
#
svg = ['<svg x="0" y="0" viewBox="0 0 128 128" width="512" height="512">']
svg.append(f'<rect x="0" y="0" width="128" height="128" fill="#ffffff" />')
cells   = rt.isedgarVoronoi(circles, [(0,0), (0,128), (128,128), (128,0)], use_circle_radius=False)
#svg.append('<rect x="23" y="60" width="10" height="10" fill="none" stroke="#000000" stroke-width="0.5" />')
def distanceToCircle(_xy_, i):
    return sqrt((circles[i][0]-_xy_[0])**2 + (circles[i][1]-_xy_[1])**2)
for box in [(0,128,0,128)]:
    x_l, x_r, y_t, y_b = box
    for x in range(x_l, x_r):
        for y in range(y_t, y_b):
            closest_circle_i = 0
            closest_d        = distanceToCircle((x,y), 0)
            for i in range(1, len(circles)):
                d = distanceToCircle((x,y), i)
                if d < closest_d:
                    closest_d        = d
                    closest_circle_i = i
            #svg.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{rt.co_mgr.getColor(closest_circle_i)}" stroke="none" stroke-width="0.5" />')
for cell in cells:
    d = f'M {cell[0][0]} {cell[0][1]} '
    for pt in cell[1:]: d += f'L {pt[0]} {pt[1]} '
    d += 'Z'
    _color_ = '#000000' # rt.co_mgr.getColor(str(cell))
    svg.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="0.2" />')
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" fill="none" />')
    svg.append(f'<circle cx="{cx}" cy="{cy}" r="1" stroke="#000000" fill="#000000" />')
svg.append('</svg>')
#rt.tile([''.join(svg)]) # point version

In [ ]:
#
# Circle Version
#
cells   = rt.isedgarVoronoi(circles, [(0,0), (0,128), (128,128), (128,0)], use_circle_radius=True, merge_threshold=1.0)
def distanceToCircle(_xy_, i):
    return sqrt((circles[i][0]-_xy_[0])**2 + (circles[i][1]-_xy_[1])**2) - circles[i][2]
for box in [(0,128,0,128)]:
    x_l, x_r, y_t, y_b = box
    for x in range(x_l, x_r):
        for y in range(y_t, y_b):
            closest_circle_i = 0
            closest_d        = distanceToCircle((x,y), 0)
            for i in range(1, len(circles)):
                d = distanceToCircle((x,y), i)
                if d < closest_d:
                    closest_d        = d
                    closest_circle_i = i
            #svg.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{rt.co_mgr.getColor(closest_circle_i)}" stroke="none" stroke-width="0.5" />')
svg_voronoi = []
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    cell = cells[i]
    d = f'M {cell[0][0]} {cell[0][1]} '
    for pt in cell[1:]: d += f'L {pt[0]} {pt[1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="1" />')
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="#000000" stroke-width="0.1" />')

seen = set()
for i in range(len(circles)):
    _poly_ = cells[i]
    for j in range(len(_poly_)):
        _xy_ = _poly_[j]
        if _xy_ not in seen:
            seen.add(_xy_)
            svg_voronoi.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + random.random()}" stroke="#000000" stroke-width="0.1" fill="none" />')

overlapping_segments = {}
poly_connects        = {'__fm__':[], '__to__':[]}
for i in range(len(cells)):
    cell_i = cells[i]
    for j in range(len(cells)):
        if j < i: continue
        cell_j = cells[j]
        for ic in range(len(cell_i)):
            edge_i = (cell_i[ic],cell_i[(ic+1)%len(cell_i)])
            for jc in range(len(cell_j)):
                edge_j = (cell_j[jc],cell_j[(jc+1)%len(cell_j)])
                if rt.segmentsOverlap(edge_i, edge_j, eps=0.4): ### !!! EPS HAD TO BE MODIFIED FOR THIS SCALE
                    poly_connects['__fm__'].append(i), poly_connects['__to__'].append(j)
                    if (i,j) not in overlapping_segments: overlapping_segments[(i,j)] = [] 
                    overlapping_segments[(i,j)].append((edge_i, edge_j))
df_poly_connects = pl.DataFrame(poly_connects)
g_poly_connects  = rt.createNetworkXGraph(df_poly_connects, [('__fm__','__to__')])
pos              = {}
for i in range(len(circles)): pos[i] = circles[i][0:2]

rt.tile([''.join(svg_header) + ''.join(svg_circles) + ''.join(svg_voronoi) + ''.join(svg_footer), 
         rt.link(df_poly_connects, [('__fm__','__to__')], pos, draw_labels=True)], spacer=20)

In [ ]:
tiles = []
for _cycle_ in nx.simple_cycles(g_poly_connects, 4):
    if len(_cycle_) >= 4:
        found_segments = set()
        for i in _cycle_:
            for j in _cycle_:
                if (i,j) in overlapping_segments:
                    for _pair_ in overlapping_segments[(i,j)]:
                        found_segments.add(_pair_[0]), found_segments.add(_pair_[1])
        tiles.append(renderSegments(found_segments, _cycle_))
rt.tile(tiles, spacer=16)

In [ ]:
def createTilesForTwoCells(cell_i, cell_j):
    _tiles_, _segments_found_ = [], set()
    for i in range(len(cells[cell_i])):
        _segment_i_ = (cells[cell_i][i], cells[cell_i][(i+1)%len(cells[cell_i])])
        for j in range(len(cells[cell_j])):
            _segment_j_ = (cells[cell_j][j], cells[cell_j][(j+1)%len(cells[cell_j])])        
            if rt.segmentsOverlap(_segment_i_, _segment_j_, eps=0.4):
                _tiles_.append(renderSegments([_segment_i_, _segment_j_], [cell_i,cell_j]))
                _segments_found_.add(_segment_i_), _segments_found_.add(_segment_j_)
    return _tiles_, _segments_found_
def createTilesForACycle(_cycle_):
    _tiles_, _all_segments_seen_ = [], set()
    for c in range(len(_cycle_)):
        i, j = _cycle_[c], _cycle_[(c+1)%len(_cycle_)]
        _subtile_, _added_segments_ = createTilesForTwoCells(i,j)
        _tiles_.append(rt.tile(_subtile_, spacer=16))
        _all_segments_seen_ = _all_segments_seen_.union(_added_segments_)
    _tiles_.append(renderSegments(list(_all_segments_seen_), _cycle_))
    return _tiles_, _all_segments_seen_
_cycle_ = [3,6,1,5]
_all_tiles_, _all_segments_ = createTilesForACycle(_cycle_)
#rt.tile(_all_tiles_, horz=False, spacer=16)
rt.tile([renderSegments(_all_segments_, _cycle_)], spacer=16)

In [ ]:
svg_closest_circle = []

x_inc, y_inc = 0.25, 0.25
x = 30.0
while x < 60.0:
    y = 50.0
    while y < 90.0:
        circle_i = _cycle_[0]
        closest_circle_i, closest_circle_d = 0, rt.segmentLength(((x,y), circles[circle_i][0:2])) - circles[circle_i][2]
        for circle_i in _cycle_[1:]:
            d = rt.segmentLength(((x,y), circles[circle_i][0:2])) - circles[circle_i][2]
            if d < closest_circle_d:
                closest_circle_i, closest_circle_d = circle_i, d    
        svg_closest_circle.append(f'<rect x="{x}" y="{y}" width="{x_inc}" height="{y_inc}" fill="{rt.co_mgr.getColor(closest_circle_i)}" stroke="none" />')
        y += y_inc
    x += x_inc
for _segment_ in _all_segments_:
    x0,y0 = _segment_[0]
    x1,y1 = _segment_[1]
    svg_closest_circle.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="0.1" />')
for circle_i in _cycle_:
    cx,cy,r = circles[circle_i]
    _color_ = rt.co_mgr.getColor(circle_i)
    svg_closest_circle.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" stroke-width="0.1" fill="none" />')
    svg_closest_circle.append(f'<circle cx="{cx}" cy="{cy}" r="0.1" stroke="#000000" fill="#000000" />')
# rt.tile([''.join(svg_header) + ''.join(svg_closest_circle) + ''.join(svg_footer)]) # fairly expensive render

In [ ]:
svg_closest_circles = []
for i in range(len(cells)):
    poly = cells[i]
    for j in range(len(poly)):
        _xy_  = poly[j]
        d_min = 1e9
        for k in range(len(circles)):
            circle              = circles[k]
            d                   = rt.segmentLength((_xy_, circle[:2])) - circle[2]
            if d < d_min: d_min = d
        for k in range(len(circles)):
            circle              = circles[k]
            d                   = rt.segmentLength((_xy_, circle[:2])) - circle[2]
            if abs(d - d_min) < 1.0:
                svg_closest_circles.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{circle[0]}" y2="{circle[1]}" stroke-width="0.5" stroke="{rt.co_mgr.getColor(k)}" />')

rt.tile([''.join(svg_header) + ''.join(svg_closest_circles) + ''.join(svg_footer),
         ''.join(svg_header) + ''.join(svg_circles) + ''.join(svg_voronoi) + ''.join(svg_footer)], spacer=16)